# K2496 example

Quantify K2496, a Ba, Ti, Si and O containing engineered glass.

In [ ]:
# Load DrWatson project manager and enable custom environment
using DrWatson
@quickactivate "HyperspectraWithNeXL"
# Load 3rd party libraries for plotting and tabulation
using Gadfly, DataFrames
# Load the necessary NeXL libraries
using Revise
using NeXLSpectrum


path = joinpath(datadir(),"exp_raw","K2496")
# Read the spectra from disk
k2496 = loadspectrum.(joinpath(path,"K2496_$(i).msa") for i in 1:3)
benitoite = loadspectrum(joinpath(path,"Benitoite std.msa"))
# Plot them using a method of Gadfly.plot(...) specialized for Spectrum items
set_default_plot_size(8inch,3inch)
plot(k2496..., benitoite, klms=[ n"O", n"Si", n"Ti", n"Ba" ], xmax=6.0e3)

Define a BasicEDS detector object with the properties of the measurement device.  Then apply the detector to the measured spectra.

In [ ]:
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
         Dict(KShell=>n"B", LShell=>n"Ca", MShell=>n"Cs"))
k2496 = map(s->apply(s,det), k2496)
benitoite = apply(benitoite,det) 
det

Compute the filtered reference spectra from the fitting standards.  In this case, we are using "Sanbornite std.msa" for the B L-family, "BaCl2 std.msa" for the Ba M-lines, "MgO std.msa" for O and pure elements for Si and Ti. 

In [ ]:
refs = references( [
  reference(n"Si", joinpath(path,"Si std.msa"), mat"Si"),
  reference(n"Ba", joinpath(path,"Sanbornite std.msa"), mat"BaSi2O5"),  # Only L-lines are clear
  reference(n"Ba", joinpath(path,"BaCl2 std.msa"), mat"BaCl2"), # Picks up the M-lines
  reference(n"Ti", joinpath(path,"Ti std.msa"), mat"Ti"),
  reference(n"O", joinpath(path,"MgO std.msa"), mat"MgO") ], det)
ENV["columns"]=400
asa(DataFrame, refs)

Fit (fit_spectra(...)) and then matrix correct (quantify(...)) the unknown spectra.

If we fit these references to the K2496 directly and then quantify, the results are ??????

In [ ]:
fs=fit_spectrum(k2496, refs)
asa(DataFrame, fs, withUnc=true)

In [ ]:
q=quantify.(fs)
asa(DataFrame, q, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))

Now we turn our attention to the quantification standard "Benitoite."  We will fit the same fitting standards to Benitoite.

In [ ]:
benitoite_std = fit_spectrum(benitoite, refs)
asa(DataFrame, [ benitoite_std ], withUnc=true)

In [ ]:
plot(benitoite_std)

In [ ]:
properties(refs.references[1].label)

In [ ]:
std_mat =  parse(Material, "BaTiSi3O9", name="Benitoite")
fs_stds = [ standardize(fsi, benitoite_std, std_mat) for fsi in fs ]
asa(DataFrame, fs_stds, withUnc=true)

Alternative syntax for when there is more than one quantification standard. 

In [ ]:
describe(asa(DataFrame, fs_stds)[:,2:end], :mean, :std)

In [ ]:
krs=kratios(benitoite_std)

From the notebook "k2496_linearOfit.ipynb", I have determined that background correcting O in K2496 and Benitoite and then taking the ratio of the number of characteristic X-ray counts gives a k-ratio of 1.055.  However, this moves the O number the wrong direction. In fact, to correct the O number would take a larger k of approximately 1.145.

In [ ]:
krs = kratios(fs_stds[1])
i = findfirst(k->n"O K-L3" in k.xrays, krs)
krs[i] = KRatio(krs[i].xrays, krs[i].unkProps, krs[i].stdProps, krs[i].standard, uv(1.055,0.0001))
quantify("k2496", optimizeks(SimpleKRatioOptimizer(1.5), krs))

In [ ]:
krs[i] = KRatio(krs[i].xrays, krs[i].unkProps, krs[i].stdProps, krs[i].standard, uv(1.145,0.0001))
quantify("k2496", optimizeks(SimpleKRatioOptimizer(1.5), krs))

In [ ]:
krs[1].unkProps

In [ ]:
q_stds = quantify.(fs_stds)
asa(DataFrame, q_stds, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))

fs is a vector of FilterFitResult items.  Plotting a FilterFitResult shows the raw spectral data (blue), the residual spectrum (red) and the rois and k-ratios in grey above the peaks.

In [ ]:
display(plot(fs[1]))

In [ ]:
display(plot(fs_stds[1]))

In [ ]:
asa(DataFrame, fs_stds[1])

The plot may be saved in various formats (including SVG, PNG and PDF).

In [ ]:
using Cairo, Fontconfig
plot(fs_stds[1]) |> SVG(joinpath(plotsdir(), "K2496 residual - indirect.svg"), 8inch, 3inch)
plot(fs_stds[1]) |> PNG(joinpath(plotsdir(), "K2496 residual - indirect.png"), 8inch, 3inch)
plot(fs_stds[1]) |> PDF(joinpath(plotsdir(), "K2496 residual - indirect.pdf"), 8inch, 3inch)

Output the quant results.  Write the results to a CSV file.

In [ ]:
df = asa(DataFrame, q_stds, nominal=parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal"))
using CSV
CSV.write(joinpath(datadir(),"exp_pro","K2496 quant - Benitoite.csv"), df)
df

In [ ]:
describe(asa(DataFrame, q_stds)[:,2:end], :mean, :std)

Tabulating a single IterationResult item provides extensive analysis details including uncertainties and matrix correction factors.

In [ ]:
asa(DataFrame, q[1])

Now let's generate a LaTeX table for the paper.

In [ ]:
using PrettyTables
tbl = vcat( (asa(DataFrame,qi) for qi in q_stds)...) 
open(joinpath(papersdir(),"Tables","K2496 results - benitoite.ltx"),"w") do io
  pretty_table(io, tbl, nosubheader=true, backend=:latex, 
            label="tbl:k2496_benitoite")
end
tbl

QED